###Import Modules

In [ ]:
from __future__ import print_function
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import numpy
import nltk
import math
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
from nltk import pos_tag, map_tag
from nltk import RegexpParser
nltk.download('universal_tagset')
from keras.models import Model
from keras.layers import Input, LSTM, Dense, RNN

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


###Data preprocessing

In [ ]:
garbage_list_input = ['í','्','े','ॅ','ी','स','ल','म','ब','प','ड','ट','क','ऐ']
garbage_list_target = ['/','\u200d','॥']

In [ ]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()


lines = open("data.txt",encoding="utf-8").read().split('\n')
for line in lines[:len(lines)]:
    flag = 0
    # if len(line.split('\t')) == 2:
    #   print(line)
    input_text, target_text = line.split('\t')
    input_text = input_text.lower()
    for t, char in enumerate(input_text):
      if char in garbage_list_input:
        flag = 1
        break
    if flag == 1:
      flag = 0
      continue

    for t, char in enumerate(target_text):
      if char in garbage_list_target:
        flag = 1
        break
    if flag == 1:
      flag = 0
      continue


    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
   
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])


###Define Hyperparameters

In [ ]:
latent_dim = 512  # Latent dimensionality of the encoding space.
num_samples = 1000  # Number of samples to train on.
batch_size = 128  # Batch size for training.
epochs = 50  # Number of epochs to train for.
layers = [512,256] # we loop LSTMCells then wrap them in an RNN layer

###Define sequence length for encoder and decoder

In [ ]:

num_encoder_tokens = len(input_token_index)
num_decoder_tokens = len(target_token_index)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)


Number of samples: 14726
Number of unique input tokens: 29
Number of unique output tokens: 71
Max sequence length for inputs: 18
Max sequence length for outputs: 21


###Prepare input data for backtransliteration

In [ ]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [ ]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.


###Creating the encoder decoder model

In [ ]:

encoder_inputs = Input(shape=(None, num_encoder_tokens))

e_outputs, h1, c1 = LSTM(latent_dim, return_state=True, return_sequences=True)(encoder_inputs) 
_, h2, c2 = LSTM(latent_dim, return_state=True)(e_outputs) 
encoder_states = [h1, c1, h2, c2]

decoder_inputs = Input(shape=(None, num_decoder_tokens))

out_layer1 = LSTM(latent_dim, return_sequences=True, return_state=True)
d_outputs, dh1, dc1 = out_layer1(decoder_inputs,initial_state= [h1, c1])
out_layer2 = LSTM(latent_dim, return_sequences=True, return_state=True)
final, dh2, dc2 = out_layer2(d_outputs, initial_state= [h2, c2])
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(final)


model = Model([encoder_inputs, decoder_inputs], decoder_outputs)



###Training the model

In [ ]:
# Run training
model.compile(optimizer='adam', loss='categorical_crossentropy')
hist = model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=100,
          validation_split=0.2)
# Save model
model.save('s2s_stack.h5')

Epoch 1/100
93/93 [==============================] - 17s 112ms/step - loss: 1.0569 - val_loss: 1.0347
Epoch 2/100
93/93 [==============================] - 9s 92ms/step - loss: 1.0312 - val_loss: 1.0239
Epoch 3/100
93/93 [==============================] - 9s 92ms/step - loss: 1.0319 - val_loss: 1.0249
Epoch 4/100
93/93 [==============================] - 9s 92ms/step - loss: 1.0254 - val_loss: 1.0273
Epoch 5/100
93/93 [==============================] - 9s 93ms/step - loss: 1.0246 - val_loss: 1.0204
Epoch 6/100
93/93 [==============================] - 9s 93ms/step - loss: 1.0172 - val_loss: 1.0271
Epoch 7/100
93/93 [==============================] - 9s 93ms/step - loss: 1.0284 - val_loss: 1.0041
Epoch 8/100
93/93 [==============================] - 9s 92ms/step - loss: 0.9814 - val_loss: 0.9624
Epoch 9/100
93/93 [==============================] - 9s 93ms/step - loss: 0.9531 - val_loss: 0.9364
Epoch 10/100
93/93 [==============================] - 9s 93ms/step - loss: 0.9251 - val_loss: 0.92

In [ ]:
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, None, 29)]   0           []                               
                                                                                                  
 input_12 (InputLayer)          [(None, None, 71)]   0           []                               
                                                                                                  
 lstm_12 (LSTM)                 [(None, None, 512),  1110016     ['input_11[0][0]']               
                                 (None, 512),                                                     
                                 (None, 512)]                                                     
                                                                                            

###Defining the encoder and decoder model for inference

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_state_input_h1 = Input(shape=(latent_dim,))
decoder_state_input_c1 = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c, 
                         decoder_state_input_h1, decoder_state_input_c1]
d_o, state_h, state_c = out_layer1(
    decoder_inputs, initial_state=decoder_states_inputs[:2])
d_o, state_h1, state_c1 = out_layer2(
    d_o, initial_state=decoder_states_inputs[-2:])
decoder_states = [state_h, state_c, state_h1, state_c1]
decoder_outputs = decoder_dense(d_o)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [ ]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c, h1, c1 = decoder_model.predict(
            [target_seq] + states_value) #######NOTICE THE ADDITIONAL HIDDEN STATES

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c, h1, c1]#######NOTICE THE ADDITIONAL HIDDEN STATES

    return decoded_sentence


##Pipeline##

In [ ]:
text1 = open("POS Hindi English Code Mixed Tweets.tsv",encoding="utf-8").read().lower()

In [ ]:
tokens= word_tokenize(text1)

In [ ]:
en_seg_list = []  #list to store all the english segments
hi_seg_list = []  #list to store all the romanized hindi segments
temp_en=[]
temp_hi=[]
switch_ctr = 0  #variable to store number of EN-HI and HI-EN switch points

In [ ]:
word_tokens = [word for word in tokens if word not in ['@','#']]

In [ ]:
for j in range(0,len(word_tokens)-3,3):

  if(word_tokens[j+1] == "en"):
    temp_en.append((word_tokens[j],word_tokens[j+2]))
    if(word_tokens[j+4]!="en"):
      switch_ctr = switch_ctr+1
      en_seg_list.append(temp_en)
      temp_en = []
      

  elif(word_tokens[j+1] == "hi"):
    #print(word_tokens[j])
    temp_hi.append((word_tokens[j],word_tokens[j+2]))
    if(word_tokens[j+4]!="hi"):
      switch_ctr = switch_ctr+1
      hi_seg_list.append(temp_hi)
      temp_hi = []
  

In [ ]:
print(en_seg_list[0])
print(hi_seg_list[0])

[('is', 'verb'), ('in', 'adp'), ('confused', 'adj'), ('state', 'noun'), ('of', 'adp'), ('mind', 'verb')]
[('hui', 'verb'), ('ya', 'conj'), ('nahi', 'part_neg'), ('is', 'det'), ('par', 'adp')]


In [ ]:
devnagri_list = []  #list to store backtransliterated hindi words in devanagari script
for k in range(len(hi_seg_list)):
  temp = []
 
  encoder_input_data = np.zeros(
    (len(hi_seg_list[k]), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')

  for j in range(len(hi_seg_list[k])):
    temp.append(hi_seg_list[k][j][0])

  for i, (input_text, target_text) in enumerate(zip(temp, target_texts)):
    for t, char in enumerate(input_text):
        print(t,char)
        encoder_input_data[i, t, input_token_index[char]] = 1.
  d=[]
  for seq_index in range(len(temp)):
    # d = []
    # Take one sequence (part of the training test)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_word = decode_sequence(input_seq)
    decoded_word = decoded_word.replace('\n','')
    d.append(decoded_word)

  devnagri_list.append(d)

In [ ]:
print(devnagri_list[0])

['हुई', 'या', 'नहीं', 'है', 'पर']


In [ ]:
corr = 0   #variable to store number of correctly identified POS Tags
total = 0  #variable to store total number of tokens

**Mapping generated tags to Universal Tagset**

In [ ]:
#For Hindi
hindi_tags = []  #list to store POS Tags of hindi words in devanagri script
for i in range(len(devnagri_list)):
  posTagged = pos_tag(devnagri_list[i])
  #print(posTagged)
  Utags = [(word, map_tag('en-ptb', 'universal', tag)) for word, tag in posTagged]
  hindi_tags.append(Utags)
  for j in range(len(devnagri_list[i])):
    total += 1
    if Utags[j][1] == (hi_seg_list[i][j][1]).upper():
      corr += 1


In [ ]:
#For English
eng_tags = []  #list to store POS Tags of english words
for i in range(len(en_seg_list)):
  en_words = [en_seg_list[i][j][0] for j in range(len(en_seg_list[i]))]
  posTagged = pos_tag(en_words)
  #print(posTagged)
  Utags = [(word, map_tag('en-ptb', 'universal', tag)) for word, tag in posTagged]
  eng_tags.append(Utags)
  for j in range(len(en_seg_list[i])):
    total += 1
    if Utags[j][1] == (en_seg_list[i][j][1]).upper():
      corr += 1

score_A = (corr/total)*100
print(score_A)

47.70199651614632


In [ ]:
for i in range(len(en_seg_list)):
  print(en_seg_list[i],":",eng_tags[i])

[('is', 'verb'), ('in', 'adp'), ('confused', 'adj'), ('state', 'noun'), ('of', 'adp'), ('mind', 'verb')] : [('is', 'VERB'), ('in', 'ADP'), ('confused', 'ADJ'), ('state', 'NOUN'), ('of', 'ADP'), ('mind', 'NOUN')]
[('contradictory', 'noun'), ('statements', 'noun')] : [('contradictory', 'NOUN'), ('statements', 'NOUN')]
[('tension', 'noun')] : [('tension', 'NOUN')]
[('former', 'adj'), ('gen', 'noun')] : [('former', 'ADJ'), ('gen', 'NOUN')]
[('anyone', 'pron'), ('heard', 'verb'), ('of', 'adp')] : [('anyone', 'NOUN'), ('heard', 'NOUN'), ('of', 'ADP')]
[('or', 'conj'), ('has', 'verb')] : [('or', 'CONJ'), ('has', 'VERB')]
[('relocated', 'verb'), ('them', 'pron'), ('to', 'part'), ('a', 'det'), ('safe', 'adj'), ('place', 'noun')] : [('relocated', 'VERB'), ('them', 'PRON'), ('to', 'PRT'), ('a', 'DET'), ('safe', 'ADJ'), ('place', 'NOUN')]
[('away', 'adv'), ('from', 'adp')] : [('away', 'ADV'), ('from', 'ADP')]
[('worldwide', 'noun')] : [('worldwide', 'NOUN')]
[('vdo', 'noun'), ('collect', 'verb')] 

In [ ]:
for i in range(len(hi_seg_list)):
  print(hi_seg_list[i],":",hindi_tags[i])

[('hui', 'verb'), ('ya', 'conj'), ('nahi', 'part_neg'), ('is', 'det'), ('par', 'adp')] : [('हुई', 'ADJ'), ('या', 'NOUN'), ('नहीं', 'NOUN'), ('है', 'NOUN'), ('पर', 'NOUN')]
[('aa', 'verb'), ('rahe', 'verb'), ('hain', 'verb')] : [('ए', 'ADJ'), ('राहे', 'NOUN'), ('हैं', 'NOUN')]
[('nahi', 'part_neg'), ('huyi', 'part'), ('to', 'part')] : [('नहीं', 'ADJ'), ('हुी', 'NOUN'), ('तो', 'NOUN')]
[('mai', 'adp'), ('kyon', 'pron_wh')] : [('मैं', 'NOUN'), ('क्यों', 'NOUN')]
[('aur', 'conj')] : [('और', 'NOUN')]
[('ne', 'adp'), ('kaha', 'pron_wh'), ('ki', 'adp'), ('pahle', 'adv'), ('kabhi', 'adv')] : [('नी', 'ADJ'), ('कहा', 'NOUN'), ('की', 'NOUN'), ('पहले', 'NOUN'), ('कभी', 'NOUN')]
[('huyi', 'part'), ('hi', 'part'), ('nahi', 'part_neg'), ('jhootha', 'part'), ('kaun', 'pron_wh')] : [('हुी', 'ADJ'), ('ही', 'NOUN'), ('नहीं', 'NOUN'), ('झूठा', 'NOUN'), ('कौन', 'NOUN')]
[('or', 'conj')] : [('ऑर', 'NOUN')]
[('kar', 'verb'), ('rahe', 'verb'), ('hain', 'verb'), ('shayad', 'noun')] : [('कर', 'ADJ'), ('राहे', '